In [ ]:
import pandas as pd

cmf = pd.read_stata("CMF/CMF_1870.dta")

capital_check = pd.read_csv("check/1870/capital_check_1870.csv")
capital_materials_value_check = pd.read_csv("check/1870/capital&materials_value_check_1870.csv")
capital_output_check = pd.read_csv("check/1870/capital&output_check_1870.csv")
hands_female_check = pd.read_csv("check/1870/hands_female_check_1870.csv")
hands_male_check = pd.read_csv("check/1870/hands_male_check_1870.csv")
hands_male_hands_female_hands_children_tot_wages_check = pd.read_csv("check/1870/hands_male&hands_female&hands_children&tot_wages_check_1870.csv")
materials_value_check = pd.read_csv("check/1870/materials_value_check_1870.csv")
materials_value_capital_check = pd.read_csv("check/1870/materials_value&capital_check_1870.csv")
materials_value_output_check = pd.read_csv("check/1870/materials_value&output_check_1870.csv")
months_active_check = pd.read_csv("check/1870/months_active_check_1870.csv")
output_check = pd.read_csv("check/1870/output_check_1870.csv")
output_capital_check = pd.read_csv("check/1870/output&capital_check_1870.csv")
output_materials_value_check = pd.read_csv("check/1870/output&materials_value_check_1870.csv")
tot_wages_check = pd.read_csv("check/1870/tot_wages_check_1870.csv")
tot_wages_output_check = pd.read_csv("check/1870/tot_wages&output_check_1870.csv")

In [ ]:
# Set a multi-index on the DataFrame using 'file_name' and 'firm_number'
cmf.set_index(["file_name", "firm_number"], inplace=True)

In [ ]:
def update_cmf(cmf, check_df, update_mapping, condition_col='transcription_error', condition_val=1):
    """
    Update the target DataFrame (cmf) using the corrections in check_df.
    
    Parameters:
      cmf (DataFrame): The DataFrame to update (with a multi-index of file_name and firm_number).
      check_df (DataFrame): The DataFrame containing corrections.
      update_mapping (dict): A dictionary mapping target column names in cmf to source column names in check_df.
      condition_col (str): The column in check_df that must equal condition_val for the row to be processed.
      condition_val: The value in condition_col that indicates a correction should be applied.
    """
    # Filter for rows that need to be updated
    updates = check_df[check_df[condition_col] == condition_val]
    for _, row in updates.iterrows():
        key = (row['file_name'], row['firm_number'])
        for target_col, source_col in update_mapping.items():
            value = row[source_col]
            # Update only if the value is not NaN
            if pd.notna(value):
                cmf.loc[key, target_col] = value

In [ ]:
# List of base variables from filenames
base_vars = [
    "capital", "capital&materials_value", "capital&output",
    "hands_female", "hands_male", "hands_male&hands_female&hands_children&tot_wages",
    "materials_value", "materials_value&capital", "materials_value&output",
    "months_active", "output", "output&capital", "output&materials_value",
    "tot_wages", "tot_wages&output"
]

# Loop and apply update_cmf
for base in base_vars:
    # Create DataFrame variable name, e.g. "capital&output" -> "capital_output_check"
    df_var_name = base.replace("&", "_") + "_check"
    df = globals().get(df_var_name)
    
    if df is not None:
        update_mapping = {col: f"correct_{col}" for col in base.split("&")}
        update_cmf(cmf, df, update_mapping)
    else:
        print(f"Warning: {df_var_name} not found.")

In [ ]:
# save the updated DataFrame to a new Stata file
cmf.reset_index(inplace=True)
cmf.to_stata("CMF/CMF_1870_updated.dta", write_index=False)